In [12]:
from elasticsearch import Elasticsearch
import statsapi

In [3]:
es = Elasticsearch(hosts='http://localhost', port=9200)
es = Elasticsearch()

#### Create an index

In [9]:
es.indices.create(index='first_index')

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'first_index'}

In [10]:
es.indices.exists(index='first_index')

True

In [18]:
games = statsapi.schedule(start_date='01/01/2021', end_date='04/29/2021')
for game in games:
    box = statsapi.boxscore_data(game.get('game_id'), timecode=None)
    game['home_batters'] = box.get('homeBatters')
    game['away_batters'] = box.get('awayBatters')
    for x in game['home_batters']:
        if x['personId'] != '0' and x['ab'] != 'AB':
            if int(x['ab']) == 0:
                x['game_ba'] = 'N/A'
            else:
                x['game_ba'] = str(round(int(x['h'])/int(x['ab']), 3))
    for x in game['away_batters']:
        if x['personId'] != '0' and x['ab'] != 'AB':
            if int(x['ab']) == 0:
                x['game_ba'] = 'N/A'
            else:
                x['game_ba'] = str(round(int(x['h'])/int(x['ab']), 3))

    es.index(index='mlb_2021',id=game_id,body=game)

In [17]:
es.indices.delete(index='games')

{'acknowledged': True}